# Pipeline

Outil généraliste servant à chaîner et encapsuler plusieurs étapes de traitement dans un flux d'apprentissage automatique dans un seul object Python.

Outil très util pour éviter la fuite d'information lors de la valisation croisée et la sélection de paramètres par recherche de grille.

Pipeline permet d'écrire du code de manière plus succincte.

In [15]:
# imports
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge 
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, train_test_split

# Simple example

In [2]:
# Load dataset
cancer = load_breast_cancer()

# Split data
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=42)

# Normalize train data
scaler         = MinMaxScaler().fit(X_train)
X_train_scaler = scaler.transform(X_train)

# Normalize test data
X_test_scaler = scaler.transform(X_test)

# Model
svm = SVC()

# Fit the model
svm.fit(X_train_scaler, y_train)

# Scoring
print(f"Train Score: {svm.score(X_train_scaler, y_train):.5f}")
print(f"Test Score:  {svm.score(X_test_scaler, y_test):.5f}")

Train Score: 0.98357
Test Score:  0.97902


### Fuite de données

```Python
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=5)
grid.fit(X_train_scaler, y_train)
```
Nous nous sommes servis de tous les données d'apprentissage recalibrées pour la recherche sur grille.

Pour chaque division dans la validation croisée, une certaines partie du jeu de données d'apprentissage original va être déclarée comme étant TRAIN set et une autre comme étant TEST set de cette division.

Dans ce cas $\Rightarrow$ Fuite d'information

Pour éviter ce problème, la division du jeu de données au cours de la validation croisée devrait être effectuée avant tout prétraitement $\Rightarrow$ Utiliser un pipeline

In [3]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1, 1, 10]}

grid = GridSearchCV(SVC(), param_grid=param_grid, cv=5)
# <!> Big mistake 
grid.fit(X_train_scaler, y_train)
print(f"Best cross validation accuracy : {grid.best_score_}")
print(f"Best param : {grid.best_params_}")
print(f"Test Accuracy score : {grid.score(X_test_scaler, y_test)}")

Best cross validation accuracy : 0.9788508891928865
Best param : {'C': 1, 'gamma': 1}
Test Accuracy score : 0.9790209790209791


# Pipeline

Pipeline pour exprimer le flux de travail d'apprentissage.

L'object Pipeline prend une liste d'étape et chaque étape est un tuple.

La classe _Pipeline_ n'est pas restreinte au prétraitement et à la classification.

Les conditions à respecter pour les estimateurs d'un pipeline sont:

- Toutes les étapes (sauf la dernière) doivent comprendre les méthodes _fit_ et _transform_:
    - _transform_ pour qu'ils soient capables de produire une nouvelle représentation des données qui pourra être utilisée dans l'étape suivante
- La derniere étape doit comprendre de la fonction _fit_ mais pas forcement de la fonction _predict_

Au cours de l'appel à Pipeline.fit, le pipeline appelle _fit_ puis _transform_ ou simplement _fit_transform_ sur chaque étape.

In [4]:
# Create pipeline
pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())])

# Fit pipeline
pipe.fit(X_train, y_train)

# Score 
print(f"Test Score: {pipe.score(X_test, y_test):.5f}")

# Steps
pipe.steps

Test Score: 0.97902


## Accéder aux attributs des étapes

In [6]:
pipe.named_steps

{'scaler': MinMaxScaler(), 'svm': SVC()}

In [7]:
pipe.named_steps['svm']

SVC()

In [8]:
pipe.named_steps['svm'].C

1.0

## Pipeline dans des recherches sur grill

La syntaxe à employer pour définir une grille dans un pipeline consiste à spécifier :

Pour chaque paramètre **nom de l'étape __ non du paramètre exact concerné**.

_MinMaxScaler_ est réajusté uniquement pour les données d'apprentissage et donc aucune information ne fuite du jeu de test.



## Method 1 : Pipeline (long)

In [9]:
param_grid = {'svm__C': [0.001, 0.01, 0.1, 1],
              'svm__gamma': [0.001, 0.01, 0.1, 1]}

pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())])

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)

grid.fit(X_train, y_train)

print(f"Best cross validation accuracy: {grid.best_score_:.5f}")
print(f"Test set score: {grid.score(X_test, y_test):.5f}")
print(f"Best param validation accuracy: {grid.best_params_}")
# Dans notre cas, grid.best_estimator_ est un pipeline avec 2 étapes
print(f"Best estimator:\n{grid.best_estimator_}")

Best cross validation accuracy: 0.96944
Test set score: 0.97902
Best param validation accuracy: {'svm__C': 1, 'svm__gamma': 1}
Best estimator:
Pipeline(steps=[('scaler', MinMaxScaler()), ('svm', SVC(C=1, gamma=1))])


In [10]:
grid.best_estimator_.named_steps["svm"]

SVC(C=1, gamma=1)

In [11]:
grid.best_estimator_.named_steps["svm"].C

1

## Method 2 : Make_pipeline (short)

In [12]:
# MinMaxScaler ont été renommée, mais il est préférable d'utiliser la construction Pipeline 
# avec des noms explicites afin de rendre chaque étape plus explicite
pipe = make_pipeline(MinMaxScaler(), PCA(n_components=2), MinMaxScaler(), verbose=True)

pipe.fit(X_train, y_train)

print(f"Best cross validation accuracy: {grid.best_score_:.5f}")
print(f"Test set score: {grid.score(X_test, y_test):.5f}")
print(f"Best param validation accuracy: {grid.best_params_}")
# Dans notre cas, grid.best_estimator_ est un pipeline avec 2 étapes
print(f"Best estimator:\n{grid.best_estimator_}")
# Steps
pipe.steps

[Pipeline] .... (step 1 of 3) Processing minmaxscaler-1, total=   0.0s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.0s
[Pipeline] .... (step 3 of 3) Processing minmaxscaler-2, total=   0.0s
Best cross validation accuracy: 0.96944
Test set score: 0.97902
Best param validation accuracy: {'svm__C': 1, 'svm__gamma': 1}
Best estimator:
Pipeline(steps=[('scaler', MinMaxScaler()), ('svm', SVC(C=1, gamma=1))])


[('minmaxscaler-1', MinMaxScaler()),
 ('pca', PCA(n_components=2)),
 ('minmaxscaler-2', MinMaxScaler())]

In [13]:
param_grid = {'ridge__alpha': [0.001, 0.01, 0.1, 1]}

pipe = make_pipeline(MinMaxScaler(), Ridge(), verbose=False)

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)

grid.fit(X_train, y_train)

print(f"Best cross validation accuracy: {grid.best_score_:.5f}")
print(f"Test set score: {grid.score(X_test, y_test):.5f}")
print(f"Best param validation accuracy: {grid.best_params_}")
print(f"Best estimator:\n{grid.best_estimator_}")

Best cross validation accuracy: 0.70815
Test set score: 0.75317
Best param validation accuracy: {'ridge__alpha': 0.1}
Best estimator:
Pipeline(steps=[('minmaxscaler', MinMaxScaler()), ('ridge', Ridge(alpha=0.1))])


# Effectuer une recherche sur grill pour trouver quel modèle utiliser

In [38]:
param_grid = [
    # Model 1
    {'classifier': [SVC()], 'preprocessing': [StandardScaler(), None],
    'classifier__C': [.001, .01, .1, 1., 1.],
    'classifier__gamma': [.001, .01, .1, 1., 1.]},
    # Model 2
    {'classifier': [RandomForestClassifier()], 'preprocessing': [None],
    'classifier__n_estimators': [100],
    'classifier__max_features': [1, 2, 3]}]

# Load dataset
cancer = load_breast_cancer()

# Split data
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=42)

pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', SVC())])
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)

grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid=[{'classifier': [SVC(gamma=0.01)],
                          'classifier__C': [0.001, 0.01, 0.1, 1.0, 1.0],
                          'classifier__gamma': [0.001, 0.01, 0.1, 1.0, 1.0],
                          'preprocessing': [StandardScaler(), None]},
                         {'classifier': [RandomForestClassifier()],
                          'classifier__max_features': [1, 2, 3],
                          'classifier__n_estimators': [100],
                          'preprocessing': [None]}])

In [ ]:
print(f"Best cross validation accuracy: {grid.best_score_:.5f}")
print(f"Test set score: {grid.score(X_test, y_test):.5f}\n")
print(f"Best param validation accuracy:\n{grid.best_params_}\n")
print(f"Best estimator:\n{grid.best_estimator_}")